In [1]:
import sys  
sys.path.insert(0, '../OMPbenders/')
%matplotlib widget

In [2]:
import pandas as pd
import gurobipy   as      gp
from   gurobipy   import GRB
from MasterProblemOMP import MasterProblem
from SubProblemOMP import SubProblem
from plotResults import plotResults

Se cambió la definición de rho_v (el 1- no va), además se redifinió self.numberOfConstraints (se definió self.intNumberOfConstraints para calcular el óptimo). Se cambió como se calcula numberOfInvaiableBlocks

AL utilizar ql_t en 0.15 la solución objetivo se convierte en 0.0, estamos usando Ton * %Cu como ql, ¿Esta bien???, como se aplica ql, sobre cada bloque o sobre la suma de estos?, hay que cambiar la función de OMPsubproblem en función de esto 

In [3]:
#path = "C:/Users/willi/OneDrive/Escritorio/Magister/Tesis-Magister/Database/integratedModel/" #Notebook
path = "/home/williams/Tesis-Magister/Databases/"
#path = "C:/Users/Williams Medina/Desktop/Tesis Magister/Tesis-Magister/ThesisCode/MainCode/Databases/integratedModel/" #Desktop
undergroundDatabaseName = 'Modelo_F_OG.xlsx'
#openPitDatabaseName = 'Modelo_F_OG.xlsx'
openPitDatabaseName = 'Modelo_F_OG_4_4_4.xlsx'

In [4]:
if undergroundDatabaseName == openPitDatabaseName:
    undergroundMineDataframe = pd.read_excel(path + undergroundDatabaseName, engine="openpyxl") #Notebook
    openPitDataframe = undergroundMineDataframe
else:
    undergroundMineDataframe = pd.read_excel(path + undergroundDatabaseName, engine="openpyxl") #Notebook
    openPitDataframe = pd.read_excel(path + openPitDatabaseName, engine="openpyxl") #Notebook

In [5]:
class Main:
    def __init__(self, undergroundMineDataframe, openPitDataframe):
        self.openPitDataframe = openPitDataframe
        self.undergroundMineDataframe = undergroundMineDataframe
        self.numberOfPeriods = 5
        self.epsilon = 100000
        self.safetyLevel = 15
        self.colHeight = 300#300 max 630
        self.minColHeight = 0.40
        
    def execute(self):
        self.createModels()
        self.setMasterProblemCrownPillarHeights()
        self.getResults()
    
    def createModels(self):
        self.createSubProblem()
        self.createMasterProblem()

    def createSubProblem(self):
        self.SubProblem = SubProblem(self.openPitDataframe, self.numberOfPeriods, self.safetyLevel)
        self.SubProblem.setParameters()

    def createMasterProblem(self):
        self.MasterProblem = MasterProblem(self.undergroundMineDataframe, self.numberOfPeriods, self.colHeight, self.minColHeight)
        self.MasterProblem.setParameters()
        
    def setMasterProblemCrownPillarHeights(self):
        self.MasterProblem.V, self.MasterProblem.rho_v = self.SubProblem.V, self.SubProblem.rho_v
        self.MasterProblem.B_v = self.SubProblem.B_v
    
    def calculateInvaiableBlocksWithSafetyLevel(self, selectedHeight):
        heightWithSafetyLevel = selectedHeight + self.safetyLevel
        if heightWithSafetyLevel not in self.MasterProblem.B_v.keys():
            closestHeight = next((height for height in sorted(self.MasterProblem.V) if height >= heightWithSafetyLevel), None)
            if closestHeight == None:
                return len(self.SubProblem.openPitBlocks)
            return len(self.MasterProblem.B_v[closestHeight])
        return len(self.MasterProblem.B_v[heightWithSafetyLevel])

    def getResults(self):
        it = 0
        self.MasterProblem.setModel()
        currentTheta = -1000000000
        while True:
            it += 1
            self.estimatedW_v, self.estimatedTheta = self.MasterProblem.optimize()
            numberOfInvaiableBlocks = self.calculateInvaiableBlocksWithSafetyLevel(self.MasterProblem.selectedHeight)
            self.subProblemObjValue, self.pi_vb = self.SubProblem.execute(numberOfInvaiableBlocks, isFinalIteration = False)
            print(it, "Objective Value =", self.MasterProblem.undergroundModel.objVal- self.MasterProblem.theta.X + self.subProblemObjValue)
            print("w_v: {}".format(self.MasterProblem.selectedHeight ))
            print("Master theta: {}".format(self.MasterProblem.theta))
            print("Current theta: {}".format(currentTheta))
            print(self.MasterProblem.selectedHeight,numberOfInvaiableBlocks)

            
            if abs(self.MasterProblem.theta.X - currentTheta) <= self.epsilon:
                objVal = self.MasterProblem.undergroundModel.objVal- self.MasterProblem.theta.X + self.subProblemObjValue
                self.subProblemObjValue, self.pi_vb = self.SubProblem.execute(numberOfInvaiableBlocks, isFinalIteration = True) #esto se cambia si queremos poner toposort
                currentTheta = self.estimatedTheta.X
                self.estimatedW_v, self.estimatedTheta = self.MasterProblem.optimize()
                print("-----------------------------Valor óptimo encontrado-----------------------------")
                print("Optimal Crown Pillar height: {}".format(self.MasterProblem.selectedHeight ))
                print(self.estimatedW_v)
                print("Optimal ObjVal: {}".format(self.MasterProblem.undergroundModel.objVal))
 
                toposort = self.MasterProblem.undergroundModel.objVal- self.MasterProblem.theta.X + self.subProblemObjValue
                print("Optimal Theta: {}".format(self.MasterProblem.theta.X))
                print("Optimal ObjVal relaxed: {}".format(objVal))
                print("Master problem obj val: {}".format(self.MasterProblem.undergroundModel.objVal))
                print("Optimal ObjVal TopoSort: {}".format(toposort))
                print(f"GAP: {100 * (objVal-toposort)/objVal}")
                print(self.MasterProblem.selectedHeight,numberOfInvaiableBlocks)
                #print("Current theta: {}".format(currentTheta))
                break
            

            currentTheta = self.estimatedTheta.X
            #self.MasterProblem.undergroundModel.addConstr(self.MasterProblem.theta <= self.subProblemObjValue + gp.quicksum(gp.quicksum(self.pi_vb[t] * len(self.MasterProblem.B_v[v]) * (self.estimatedW_v[v] - self.MasterProblem.w_v[v]) for t in self.MasterProblem.t_S) for v in self.MasterProblem.V))

            self.MasterProblem.undergroundModel.addConstr(self.MasterProblem.theta <= self.subProblemObjValue + gp.quicksum(gp.quicksum(self.pi_vb[t] * numberOfInvaiableBlocks * (self.estimatedW_v[v] - self.MasterProblem.w_v[v]) for v in self.MasterProblem.V) for t in self.MasterProblem.t_S))
            print()
            print()

In [12]:
print(main.pi_vb)

{0: 4096.0, 1: 4096.0, 2: 4096.0, 3: 4096.0, 4: 4096.0}


In [6]:
main = Main(undergroundMineDataframe, openPitDataframe)
main.execute()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-02
Set parameter TimeLimit to value 3600
Se optimizó el subproblema con 11760 bloques infactibles
Running OMP version: master 1.8.5 d7d16093 2022-11-16 20:03:46 -0300
Invoked using: bin/OMP_SOLVER_C1290 ../FilesToExecuteOmpOpenPit/files/openPit.blocks ../FilesToExecuteOmpOpenPit/files/openPit.prec ../FilesToExecuteOmpOpenPit/files/openPit.prob ../FilesToExecuteOmpOpenPit/params/dbs_duals.params
license to sign [DEFAULTclient-signatureAFhG-4bsA-AAA=lic_ver200valid-to2023-12-05]
LICENSE OK!
BLOCKS FILE: ../FilesToExecuteOmpOpenPit/files/openPit.blocks
PREC FILE: ../FilesToExecuteOmpOpenPit/files/openPit.prec
PROB FILE: ../FilesToExecuteOmpOpenPit/files/openPit.prob
PARAMS FILE: ../FilesToExecuteOmpOpenPit/params/dbs_duals.params
OR Solver: key created. The key is: openPit_openPit_dbs_duals
STATS_INPUT_NAME openPit_openPit_dbs_duals
OR Solver: loaded params (params file) OK.
STATS_PARAM_STOCHASTIC_MODE

In [7]:
main.MasterProblem.undergroundModel.objVal-main.MasterProblem.theta.X , main.subProblemObjValue, main.MasterProblem.theta.X

(2052528067.1085415, 0.0, 86744000.0)

In [8]:
len(main.MasterProblem.drawpoint)/main.numberOfPeriods

33.6

In [9]:
main.subProblemObjValue

0.0

In [10]:
#main2 = Main(undergroundMineDataframe, undergroundMineDataframe)
#main2.execute()

In [11]:
main2.MasterProblem.rho_v[1000]

NameError: name 'main2' is not defined

In [ ]:
main2.MasterProblem.undergroundModel.objVal-main2.MasterProblem.theta.X , main2.subProblemObjValue, main2.MasterProblem.theta.X

NameError: name 'main2' is not defined

In [ ]:
len(main.SubProblem.B_v[1225])

9408

5 desagregado
Optimal ObjVal: 2742740809.108471
Optimal Theta: 682920000.0
Optimal ObjVal TopoSort: 2742740809.108471
3.2 segs


10 desagregado
Optimal ObjVal: 2268360094.125119
Optimal Theta: 561370000.0
Optimal ObjVal TopoSort: 2268360094.125119
15.4 segs


15 desagregado 
Optimal ObjVal: 1906146810.5342412
Optimal Theta: 469290000.0
Optimal ObjVal TopoSort: 1906146810.5342412 
1min 12 segs



20 desagregado Optimal ObjVal: 1624948384.8196843
Optimal Theta: 399730000.0
Optimal ObjVal TopoSort: 1624948384.8196843
GAP: 0.0 
8min 19.9 segs

10 desagregado

Optimal ObjVal: 2343184937.4682417
Optimal Theta: 640110000.0
Optimal ObjVal TopoSort: 2329594937.4682417
GAP: 0.5799798292781656

5 desagregado

Optimal ObjVal: 2840469882.9332986
Optimal Theta: 781890000.0
Optimal ObjVal TopoSort: 2830099882.9332986
GAP: 0.3650804418771411

15 agregado

Optimal Crown Pillar height: {745: 1.0, 785: 0.0, 825: 0.0, 865: 0.0, 905: 0.0, 945: 0.0, 985: 0.0, 1025: 0.0, 1065: 0.0, 1105: 0.0, 1145: 0.0, 1185: 0.0, 1225: 0.0, 1265: 0.0, 1305: 0.0}
Optimal ObjVal: 1903625419.951932
Optimal Theta: 469290000.0
Optimal ObjVal TopoSort: 1884355419.951932
GAP: 1.0122789808347161

------------------------

VALORES PARA BLOQUES AGREGADOS, SOLUCIÓN RELAJADA


5 PERIODOS: 3.3 segs

* Optimal ObjVal: 2742740809.10847
* Optimal Theta: 682920000.0
* Optimal ObjVal relaxed: 2742740809.108471
* Master problem obj val: 2742740809.108471
* Optimal ObjVal TopoSort: 2742740809.108471

10 PERIODOS: 7.0 segs

* Optimal ObjVal: 2271941260.015317
* Optimal Theta: 561370000.0
* Optimal ObjVal relaxed: 2271941260.015317
* Master problem obj val: 2271941260.015317
* Optimal ObjVal TopoSort: 2271941260.015317
* GAP: 0.0

15 PERIODOS: 14.1 segs

* Optimal ObjVal: 1873723971.8816504
* Optimal Theta: 469290000.0
* Optimal ObjVal relaxed: 1873723971.8816504
* Master problem obj val: 1873723971.8816504
* Optimal ObjVal TopoSort: 1873723971.8816504
* GAP: 0.0

20 PERIODOS: 25.5 segs

* Optimal ObjVal: 1616265630.1377
* Optimal Theta: 399730000.0
* Optimal ObjVal relaxed: 1616265630.1377
* Master problem obj val: 1616265630.1377
* Optimal ObjVal TopoSort: 1616265630.1377
* GAP: 0.0

------------------------

VALORES PARA BLOQUES DESAGREGADOS, SOLUCIÓN RELAJADA


5 PERIODOS: 59.2 segs

* Optimal ObjVal: 2807234376.8134923
* Optimal Theta: 781890000.0
* Optimal ObjVal relaxed: 2807234376.8134923
* Master problem obj val: 2807234376.8134923
* Optimal ObjVal TopoSort: 2807234376.8134923

10 PERIODOS: 1 min 27.3 segs

* Optimal ObjVal: 2353537399.199758
* Optimal Theta: 640110000.0
* Optimal ObjVal relaxed: 2353537399.199758
* Master problem obj val: 2353537399.199758
* Optimal ObjVal TopoSort: 2353537399.199758
* GAP: 0.0

15 PERIODOS: 2 min 10.2 segs

* Optimal ObjVal: 1962744919.4705706
* Optimal Theta: 532320000.0
* Optimal ObjVal relaxed: 1962744919.4705706
* Master problem obj val: 1962744919.4705706
* Optimal ObjVal TopoSort: 1962744919.4705706
* GAP: 0.0

20 PERIODOS: 3 min 0.9 segs

* Optimal ObjVal: 1657095434.8418837
* Optimal Theta: 449440000.0
* Optimal ObjVal relaxed: 1657095434.8418837
* Master problem obj val: 1657095434.8418837
* Optimal ObjVal TopoSort: 1657095434.8418837
* GAP: 0.0

------------------------

VALORES PARA BLOQUES AGREGADOS, SOLUCIÓN ENTERA


5 PERIODOS: 3.2 segs

* Optimal ObjVal: 2742740809.108471
* Optimal Theta: 682920000.0
* Optimal ObjVal relaxed: 2742740809.108471
* Master problem obj val: 2742740809.108471
* Optimal ObjVal TopoSort: 2686060809.108471
* GAP: 2.0665459824628436

10 PERIODOS: 7.0 segs

* Optimal ObjVal: 2271941260.015317
* Optimal Theta: 561370000.0
* Optimal ObjVal relaxed: 2271941260.015317
* Master problem obj val: 2271941260.015317
* Optimal ObjVal TopoSort: 2228841260.015317
* GAP: 1.8970560884883718

15 PERIODOS: 13.9 segs
* Optimal ObjVal: 1873723971.8816504
* Optimal Theta: 469290000.0
* Optimal ObjVal relaxed: 1873723971.8816504
* Master problem obj val: 1873723971.8816504
* Optimal ObjVal TopoSort: 1854453971.8816504
* GAP: 1.0284332318515668

20 PERIODOS: 25.3 segs 

* Optimal ObjVal: 1616265630.1377
* Optimal Theta: 399730000.0
* Optimal ObjVal relaxed: 1616265630.1377
* Master problem obj val: 1616265630.1377
* Optimal ObjVal TopoSort: 1593475630.1377
* GAP: 1.4100405017001056

------------------------

VALORES PARA BLOQUES DESAGREGADOS, SOLUCIÓN ENTERA


5 PERIODOS: 3.3 segs

* Optimal ObjVal: 2807234376.8134923
* Optimal Theta: 781890000.0
* Optimal ObjVal relaxed: 2807234376.8134923
* Master problem obj val: 2807234376.8134923
* Optimal ObjVal TopoSort: 2796864376.8134923
* GAP: 0.3694027148446025

10 PERIODOS: 1 min 27.7 segs

* Optimal ObjVal: 2353537399.199758
* Optimal Theta: 640110000.0
* Optimal ObjVal relaxed: 2353537399.199758
* Master problem obj val: 2353537399.199758
* Optimal ObjVal TopoSort: 2339947399.199758
* GAP: 0.5774286826553436

15 PERIODOS: 2 min 9.8 segs

* Optimal ObjVal: 1962744919.4705706
* Optimal Theta: 532320000.0
* Optimal ObjVal relaxed: 1962744919.4705706
* Master problem obj val: 1962744919.4705706
* Optimal ObjVal TopoSort: 1941014919.4705706
* GAP: 1.1071229778478517

20 PERIODOS: 2 min 59.9 segs

* Optimal ObjVal: 1657095434.8418837
* Optimal Theta: 449440000.0
* Optimal ObjVal relaxed: 1657095434.8418837
* Master problem obj val: 1657095434.8418837
* Optimal ObjVal TopoSort: 1640575434.8418837
* GAP: 0.9969250806352201

------------------------